In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [20]:
def generate_dates():
    
    number = np.random.randint(1,125)
    if number%3 == 2:
        period = number * 7
    elif number%3 == 1:
        period = number * 7 + 3
    else:
        period = number * 7 + 6
        
    start = datetime.strptime('2021-04-07', "%Y-%m-%d")
    tournament_date = start + timedelta(days=period)

    return(tournament_date)
    

In [21]:
def generate_total_players(table_game,game_id):
    
    inventory_table = pd.read_csv('inventory_rent.csv')
    
    games_number = inventory_table.game_id.value_counts()
    availability_df = pd.DataFrame(games_number)
    
    games_table = table_game[['game_id','min_players','max_players','duration', 'max_players_in_team','min_players_in_team']]
    game_id_df = pd.DataFrame(game_id, columns=['game_id'])
    merged = game_id_df.merge(games_table, on='game_id', how='inner')
    merged_availability = merged.merge(availability_df, on='game_id', how='inner')

    total_num = []
    for i in range(len(merged_availability)):
        row = merged_availability.iloc[i]
        
        if row['count'] > 20:
            boards_num = 16
        elif row['count'] > 10 and row['count'] <= 20: 
            boards_num = int(np.ceil(row['count'] / 2))
        else:
            boards_num = row['count'] - 1
        
        if row.max_players_in_team > 1: 
            teams_per_board =  int(np.floor(row['max_players'] / row['max_players_in_team']))
            players_all_board = [teams_per_board * row.max_players_in_team for i in range(boards_num)]
            total_num.extend([np.sum(players_all_board)])

        else:
            players_per_board = [np.random.randint(row.min_players,row.max_players + 1) for i in range(boards_num)]
            total_num.extend([np.sum(players_per_board)])
            
    merged_availability['total_players_number'] = total_num
    
    return(merged_availability.total_players_number)

In [24]:
def generate_tournament():
    
    games_csv = pd.read_csv('game.csv')
    staff_csv = pd.read_csv('staff.csv')

    name = ['Cosmic Entertainment', 'Battlefields of Bonaparte', 'The Kobolds', 'Gaming Evening', 'Hamst&Furious', 'Indian Camp', 'Vietgame',
             'Thematic Contest', 'Spring Tournament', 'Star Trek Day']
    n = len(name)
    tournament_id = np.arange(1,n+1)
    game_id = [9,27,33,33,7,5,36,6,1,19]
    date = np.sort([generate_dates() for _ in range(n)])
    
    team_players_number = [games_csv.max_players_in_team[games_csv.game_id == i].values[0] for i in game_id]
    total_players_number = generate_total_players(games_csv, game_id)
    
    max_staff = staff_csv.staff_id.max()
    staff_id = [np.random.randint(1,max_staff+1) for _ in range(n)]
    
    
    tournaments_dict = {'tournament_id' : tournament_id,
                    'name' : name,
                    'date' : date,
                    'game_id' : game_id,
                    'team_players_number' : team_players_number,
                    'staff_id' : staff_id,
                    'total_players_number' : total_players_number
}
    tournaments_df = pd.DataFrame(tournaments_dict)
    tournaments_df.to_csv('tournament.csv', index = False)

In [25]:
generate_tournament()

In [26]:
pd.read_csv('tournament.csv')

,tournament_id,name,date,game_id,team_players_number,staff_id,total_players_number
0,1,Cosmic Entertainment,2021-04-17,9,1,2,16
1,2,Battlefields of Bonaparte,2021-08-21,27,1,3,11
2,3,The Kobolds,2021-09-11,33,1,3,19
3,4,Gaming Evening,2021-12-04,33,1,2,15
4,5,Hamst&Furious,2022-08-17,7,1,2,56
5,6,Indian Camp,2022-10-15,5,1,5,14
6,7,Vietgame,2023-03-07,36,1,4,16
7,8,Thematic Contest,2023-04-01,6,2,5,36
8,9,Spring Tournament,2023-05-13,1,1,2,52
9,10,Star Trek Day,2023-06-07,19,1,3,15
